# Exam Problem 

Cristina Hernández, Beatriz Jimenez, Macarena Vargas, Guillermo Ruiz


In [70]:
import pyomo.environ as pe
import pyomo.opt as po

#### Create the model 

In [71]:
model = pe.ConcreteModel()

#### Sets


${i}$: Course i = {A, B, C, D}
${j}$: Number of days j = {1, 2, 3 , 4}



In [72]:
model.courses = pe.Set(initialize=['A', 'B', 'C', 'D'])
model.days = pe.Set(initialize=[1, 2, 3, 4])

#### Parameters


${n}$ : number of days to study 

${G_{ij}}$ : grade if the student studies the course i , j days 

In [73]:
model.num_days = pe.Param(initialize = 7)

dict_grades_per_days =  {
    ('A', 1): 3, ('A', 2): 5, ('A', 3): 6, ('A', 4): 7, 
    ('B', 1): 4, ('B', 2): 5, ('B', 3): 6, ('B', 4): 9, 
    ('C', 1): 2, ('C', 2): 5, ('C', 3): 7, ('C', 4): 8, 
    ('D', 1): 5, ('D', 2): 6, ('D', 3): 7, ('D', 4): 9}

model.grades_per_days = pe.Param(model.courses, model.days, initialize = dict_grades_per_days)

#### Variables 

${x_{ij}}$: the student studies j days the course i(bool)

In [74]:
model.days_per_course = pe.Var(model.courses, model.days, within = pe.NonNegativeReals)

${p_{i}}$ : the student passes the course i(bool)

In [75]:
model.pass_course = pe.Var(model.courses,within = pe.Binary)

#### Constrains 

1)The students studies either 1, 2, 3 or four days
(equivalent to have only value of 1 in "number of days" j )

$\sum_{j}(x_{ij}) = 1$ 

2) The student only has seven days to study

$\sum_{i,j} (x_{ij})* ord(n) = 7 $ 


3) The student passes the course i if the grade assigned to the j days he studied is >= 5

$\sum_{j}(G_{ij}) * x_{ij}  \;\geq\; 5  ;  p_{i} = 1 $

$\sum_{j}(G_{ij}) * x_{ij}  < 5  ;  p_{i} = 0 $

This is rephrased as : 
$\sum_{j}(G_{ij}) * x_{ij}  \;\geq\;  5 - \varepsilon  + (M + \varepsilon ) * p_{i}$



In [76]:
eps = 1e-6 #a small positive number
M = 10 #a big positive number


def pass_course(m, i):
    return sum(m.grades_per_days[i,j] * m.days_per_course[i,j] for j in m.days) >= 5 - eps + (M + eps) * m.pass_course[i]
model.pass_course = pe.Constraint(model.courses, rule=pass_course)

'pyomo.core.base.var.IndexedVar'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.constraint.IndexedConstraint'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
ERROR: Rule failed when generating expression for Constraint pass_course with
index A: RecursionError: maximum recursion depth exceeded
ERROR: Constructing component 'pass_course' from data=None failed:
        RecursionError: maximum recursion depth exceeded


RecursionError: maximum recursion depth exceeded

####